### Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# ELT Project

## Read Country and Country Codes

In [2]:
url = 'https://www.iban.com/country-codes'

In [3]:
countries_conv = pd.read_html(url)

In [4]:
countries_conv_df = countries_conv[0]
countries_conv_df

,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,AF,AFG,4
1,Åland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16
...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876
245,Western Sahara,EH,ESH,732
246,Yemen,YE,YEM,887
247,Zambia,ZM,ZMB,894


In [5]:
countries_data = countries_conv_df[['Country','Alpha-3 code']].copy()
countries_data.head()

,Country,Alpha-3 code
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [6]:
countries_data['new_name']=countries_data['Country'].str.replace(" ","")

In [7]:
countries_data['new_name'] = countries_data['new_name'].str.lower()

## Matching Database

In [8]:
countries_data = countries_data.rename(columns={'Country':'country_name','Alpha-3 code':'country_code'})
countries_data.head()

,country_name,country_code,new_name
0,Afghanistan,AFG,afghanistan
1,Åland Islands,ALA,ålandislands
2,Albania,ALB,albania
3,Algeria,DZA,algeria
4,American Samoa,ASM,americansamoa


## Create Database Connection

In [9]:
rds_connection_string = "postgres:<pass>localhost:5432/ELT_Happiness"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
# Confirm tables
engine.table_names()

['indicators_data', 'hapiness', 'country', 'indicators']

## Load DataFrame into database

In [11]:
countries_data.to_sql(name='country', con=engine, if_exists='append', index=False)